# Initialize

## Import modules

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

mpl.rcParams['font.size'] = 14

# scientific format of axis labels
def sci_format():
    fmt = mpl.ticker.ScalarFormatter(useMathText=True)
    fmt.set_powerlimits((-2, 3))
    return (fmt)

## Define analyzer

In [ ]:
class Analyze(object):
    
    def __init__(self,
                 root=".",      # path to result data
                 x_scale = 1.0,
                 y_scale = 1.0,
                 u_scale = 1.0,
                 rotate = False, # rotate points from global (x,y) to local (x',y')
                 ref_x = 0.0,
                 ref_y = 0.0
                ):
        self = locals().pop("self")
        for name, val in locals().items():
            print("setting " + name + " to:", val)
            setattr(self, name, val)
        
        # load all data
        self.load_source()
        self.load_target()
        self.load_error()
        
        if self.rotate:
            from scipy.spatial.transform import Rotation as rotate
                
            # compute rotation matrix    
            r0 = np.sqrt(self.ref_x**2 + self.ref_y**2)
            alpha0 = np.arctan2(self.ref_x, self.ref_y)
            r = rotate.from_euler('z', alpha0)
            rot_matrix = r.as_matrix()
            
            print("alpha0 = ", alpha0)
            print("rot_matrix=", rot_matrix)
            print("ref_prime = ", r.apply(np.asarray([self.ref_x, self.ref_y, 0.0])))
            
            # rotate from global (x,y) to local (x',y')
            self.source_x_prime = self.source_x * rot_matrix[0,0] + self.source_y * rot_matrix[0,1]
            self.source_y_prime = self.source_x * rot_matrix[1,0] + self.source_y * rot_matrix[1,1]
            self.target_x_prime = self.target_x * rot_matrix[0,0] + self.target_y * rot_matrix[0,1]
            self.target_y_prime = self.target_x * rot_matrix[1,0] + self.target_y * rot_matrix[1,1]            
            
            # deduce axis range
            self.target_x_prime_lim = [self.target_x_prime.min(), self.target_x_prime.max()] 
            self.target_y_prime_lim = [self.target_y_prime.min(), self.target_y_prime.max()] 
            
    def load_source(self):
        # import points
        self.source_x, self.source_y = np.loadtxt(self.root + '/source.csv', delimiter=',', unpack=True)
        self.source_x *= self.x_scale
        self.source_y *= self.y_scale
        # deduce axis range
        self.xlim_source = [self.source_x.min(), self.source_x.max()]
        self.ylim_source = [self.source_y.min(), self.source_y.max()]
        # import field and unscale
        self.source = np.loadtxt(self.root+'/source.dat', delimiter=',', unpack=True)
        self.source *= self.u_scale

    def load_target(self):
        from scipy.spatial.transform import Rotation as R
        # import points
        self.target_x, self.target_y = np.loadtxt(self.root + '/target.csv', delimiter=',', unpack=True)
        # deduce axis range
        self.xlim_target = [self.target_x.min(), self.target_x.max()]
        self.ylim_target = [self.target_y.min(), self.target_y.max()]
        # fields
        self.remap = np.loadtxt(self.root+'/remap.dat', delimiter=',', unpack=True)

    def load_error(self):
        self.error = np.loadtxt(self.root+'/error.dat', delimiter=',', unpack=True)
        self.exact = np.loadtxt(self.root+'/exact.dat', delimiter=',', unpack=True)

# Plotting

## Verify source points

In [ ]:
# optional scaling
x_scale = 1e-6
y_scale = 1e-4
u_scale = 1e-8

# optional reference point for rotation
ref_x = 0.198179158531
ref_y = 0.980165685551

def load_source(path, 
                assign_field_value=True,
                x_scale = 1.0,
                y_scale = 1.0,
                u_scale = 1.0) :
    data = np.genfromtxt(path, delimiter=",") # read CSV file
    n_source = data.shape[0]
    u = np.zeros(n_source)

    def assign_field(p):
        x = p[:,0]
        y = p[:,1]
        x_fact = -0.0000001
        y_fact = -0.0005
        return np.exp((x_fact*x*x) + (y_fact*y*y))
        #return np.cos(5.0*y*2*np.pi/(ymax-ymin))

    if (assign_field_value) : 
        u = assign_field(data[:,:2])
        u *= u_scale
 
    x = data[:,0].copy() * x_scale
    y = data[:,1].copy() * y_scale
    return x, y, u

# load source file
x, y, u = load_source("source.csv", True, x_scale, y_scale, u_scale)

# compare with previoulsy loaded
analysis = Analyze(".", x_scale, y_scale, u_scale, False, ref_x, ref_y)
print("source_x: (min,max) = ", x.min(), x.max())
print("source_y: (min,max) = ", y.min(), y.max())
print("diff_x = ", analysis.source_x.min() - x.min(), analysis.source_x.max() - x.max())
print("diff_y = ", analysis.source_y.min() - y.min(), analysis.source_y.max() - y.max())

## Points distribution

In [ ]:
# plot source and target
fig, axes = plt.subplots(1,1, figsize=[8,6])
ax=axes
ax.set_title("source & target")
ax.scatter(analysis.source_x, analysis.source_y, s=.1, facecolor='C0', label="source")
ax.scatter(analysis.target_x, analysis.target_y, s=.1, facecolor='C1', label="target")
ax.set_xlim(analysis.xlim_target)
ax.set_ylim(analysis.ylim_target)
ax.set_xlabel(r'$x\prime$')
ax.set_ylabel(r'$y\prime$')
plt.legend(loc="upper right")

print(analysis.target_x.min(), analysis.target_y.min())
print(analysis.target_x.max(), analysis.target_y.max())

## Source field and comparison

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(2,2, figsize=[16,12])

ax=axes[0,0]
ax.set_title("source field")
sc=ax.scatter(analysis.source_x, analysis.source_y, c=analysis.source, marker='o', s=5, cmap="coolwarm")
ax.set_xlim(analysis.xlim_target)
ax.set_ylim(analysis.ylim_target)
fig.colorbar(sc, ax=ax)
#sc.set_clim(-1, 1)
ax.set_xlabel(r'$x\prime$')
ax.set_ylabel(r'$y\prime$')

ax=axes[0,1]
ax.set_title("remapped field")
sc=ax.scatter(analysis.target_x, analysis.target_y, c=analysis.remap, marker='o', s=5, cmap="coolwarm")
# ax.plot(analysis.ref_x, analysis.ref_y, 'c.', markersize=5)
ax.set_xlim(analysis.xlim_target)
ax.set_ylim(analysis.ylim_target)
fig.colorbar(sc, ax=ax)
ax.set_xlabel(r'$x\prime$')
ax.set_ylabel(r'$y\prime$')

analysis.load_error()
ax=axes[1,0]
ax.set_title("exact values")
sc=ax.scatter(analysis.target_x, analysis.target_y, c=analysis.exact, marker='o', s=5, cmap="coolwarm")
ax.set_xlim(analysis.xlim_target)
ax.set_ylim(analysis.ylim_target)
fig.colorbar(sc, ax=ax)
ax.set_xlabel(r'$x\prime$')
ax.set_ylabel(r'$y\prime$')

ax=axes[1,1]
ax.set_title("error map")
sc=ax.scatter(analysis.target_x, analysis.target_y, c=analysis.error, marker='.', s=5, cmap="coolwarm")
ax.set_xlim(analysis.xlim_target)
ax.set_ylim(analysis.ylim_target)
fig.colorbar(sc, ax=ax)
ax.set_xlabel(r'$x\prime$')
ax.set_ylabel(r'$y\prime$')

plt.savefig("fields.png")

## Remapped field

In [ ]:
%matplotlib inline
plt.subplots(1,1, figsize=[8,6])
plt.title("remapped field")
plt.imshow(analysis.remap.reshape([201,201]).T,\
           aspect="auto",cmap="coolwarm",\
           origin="upper", \
           extent=[analysis.xlim_target[0], analysis.xlim_target[1], \
                   analysis.ylim_target[0], analysis.ylim_target[1]])
#plt.clim(-1,1)
plt.colorbar()

## Exact values

In [ ]:
%matplotlib inline
analysis.load_error()
plt.subplots(1,1, figsize=[8,6])
plt.title("exact values")
plt.imshow(analysis.exact.reshape([201,201]).T,\
           aspect="auto", cmap="coolwarm")
#plt.clim(-1,1)
plt.colorbar()

analysis.source.min(), analysis.source.max()

## Error map

In [ ]:
%matplotlib inline
plt.subplots(1,1, figsize=[8,6])
plt.title("error map")
plt.imshow(analysis.error.reshape([201,201]).T,\
           aspect="auto", cmap="coolwarm")
#plt.clim(0,0.5)
plt.colorbar()